<a href="https://colab.research.google.com/github/DK-77/CE059_Dhruv_Kathrotiya/blob/main/Lab5/Lab5_Task2_CE059.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Numpy & PyTorch
import numpy as np
import torch

## Linear Regression Model using PyTorch built-ins

Let's re-implement the same model using some built-in functions and classes from PyTorch.

And now using two different targets: Apples and Oranges

In [2]:
# Imports
import torch.nn as nn

In [3]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70], [73, 67, 43], [91, 88, 64], [87, 134, 58], [102, 43, 37], [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119], 
                    [56, 70], [81, 101], [119, 133], [22, 37], [103, 119]], dtype='float32')

In [4]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print("Inputs : \n",inputs)
print("Targets : \n",targets)

Inputs : 
 tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
Targets : 
 tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples. We'll also create a DataLoader, to split the data into batches while training. It also provides other utilities like shuffling and sampling.

In [5]:
# Import tensor dataset & data loader
from torch.utils.data import TensorDataset, DataLoader

In [6]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [7]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [8]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [102.,  43.,  37.],
        [ 87., 134.,  58.],
        [ 73.,  67.,  43.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [ 22.,  37.],
        [119., 133.],
        [ 56.,  70.]])


### nn.Linear
Instead of initializing the weights & biases manually, we can define the model using `nn.Linear`.

In [9]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1734, -0.0404, -0.2772],
        [-0.4927,  0.5397,  0.1515]], requires_grad=True)
Parameter containing:
tensor([0.5129, 0.3833], requires_grad=True)


In [10]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1734, -0.0404, -0.2772],
         [-0.4927,  0.5397,  0.1515]], requires_grad=True),
 Parameter containing:
 tensor([0.5129, 0.3833], requires_grad=True)]

In [11]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ -1.4550,   7.0845],
        [ -5.0030,  12.7291],
        [ -5.8911,  38.6158],
        [  6.2051, -21.0655],
        [-10.8033,  28.7954],
        [ -1.4550,   7.0845],
        [ -5.0030,  12.7291],
        [ -5.8911,  38.6158],
        [  6.2051, -21.0655],
        [-10.8033,  28.7954],
        [ -1.4550,   7.0845],
        [ -5.0030,  12.7291],
        [ -5.8911,  38.6158],
        [  6.2051, -21.0655],
        [-10.8033,  28.7954]], grad_fn=<AddmmBackward>)

### Loss Function
Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [12]:
# Import nn.functional
import torch.nn.functional as F

In [13]:
# Define loss function
loss_fn = F.mse_loss

In [14]:
loss = loss_fn(model(inputs) , targets)
print(loss)

tensor(7166.3003, grad_fn=<MseLossBackward>)


### Optimizer
Instead of manually manipulating the weights & biases using gradients, we can use the optimizer `optim.SGD`.

In [15]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

### Train the model

We are ready to train the model now. We can define a utility function `fit` which trains the model for a given number of epochs.

In [16]:
# Define a utility function to train the model
def fit(num_epochs, model, loss_fn, opt):
   for epoch in range(num_epochs):
       for xb,yb in train_dl:
            # Generate predictions
            pred = model(xb)
            loss = loss_fn(pred,yb)
            # Perform gradient descent
            loss.backward()
            opt.step()
            opt.zero_grad()
   print('Training loss: ', loss_fn(model(inputs), targets))

In [17]:
# Train the model for 100 epochs
fit(100 , model , loss_fn, opt)

Training loss:  tensor(29.0012, grad_fn=<MseLossBackward>)


In [18]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 59.0618,  70.8362],
        [ 79.8069,  97.8883],
        [121.9840, 138.5927],
        [ 30.5373,  39.2643],
        [ 92.2646, 113.0512],
        [ 59.0618,  70.8362],
        [ 79.8069,  97.8883],
        [121.9840, 138.5927],
        [ 30.5373,  39.2643],
        [ 92.2646, 113.0512],
        [ 59.0618,  70.8362],
        [ 79.8069,  97.8883],
        [121.9840, 138.5927],
        [ 30.5373,  39.2643],
        [ 92.2646, 113.0512]], grad_fn=<AddmmBackward>)

In [19]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])